<a href="https://colab.research.google.com/github/williamlidberg/Analyses-of-Environmental-Data-2/blob/main/modules/module_5/assignment_5_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyse satellite imagery with Geemap Updated for 2025
Geemap is a Python tool that makes it easy for beginners to create interactive maps using Google Earth Engine (GEE). Google Earth Engine is like a powerful online library of satellite images and geographic data stored in the cloud. People in the field of geography and environmental science widely use it.

Google Earth Engine has tools for programming in both JavaScript and Python. While the JavaScript tools have lots of helpful documentation and an interactive editor, the Python tools have less documentation and are not as user-friendly for visualizing results.

That's where Geemap comes in. Geemap is a Python package that was made to bridge this gap. It uses ipyleaflet and ipywidgets to help you analyze and visualize data from Google Earth Engine in a more beginner-friendly way. You can easily work with geographic datasets and create interactive maps right in a Jupyter notebook. It's a handy tool for anyone wanting to explore and understand Earth observation data without diving too deep into complex coding.

# Installation
Geemap is preinstalled with google colab but you can install it using Pypip or anaconda as well:\
pip install geemap\
conda install geemap -c conda-forge

Read the documentation for more info regarding installation on different systems: https://courses.geemap.org/geemap_intro/02_installation/#key-features

### Task 1 Setup
To use geemap, you must first sign up for a Google Earth Engine account. You cannot use Google Earth Engine unless your application has been approved. Once you receive the application approval email. The next stepp is to run the code below and grant access for google colab to use your google account and google earth engine. If everything works as planned you should see an interractive map.

In [ ]:
# Authenticate and initialize Google Earth Engine API
import ee
ee.Authenticate()
ee.Initialize(project='ee-williamlidberg') # change ee-williamlidberg to your own user

To check if everything is working run these lines. They should output a map.

In [ ]:
import geemap
Map = geemap.Map()
Map

If you want a different basemap you can select one from the list.

In [ ]:
for basemap in geemap.basemaps.keys():
    print(basemap)

If you want to center the map on somewhere specific you can set the coordinates and zoom level with .setCenter(). Note that the coordinates are in long/lat. When you right click in google maps you get the coordinates in lat/long. If you want to translate the coordiantes from google maps simply switch the numbers around so 64.2, 20 becomes 20, 64.2

In [ ]:
Map = geemap.Map()
Map.add_basemap('CartoDB.DarkMatter')
Map.setCenter(20, 64.2, 5) # This is the coordinates in long/lat and the zoom level.
Map

# Working with image collections
Google earth engine comes with alot of large image series that you can use for your projects. The [sentinel 2 constellation](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/news/-/asset_publisher/Ac0d/content/sentinel-2-images-the-globe-every-5-days;jsessionid=2BC4823847466509FA382BB7ED954C47.jvm1?redirect=https%3A%2F%2Fsentinels.copernicus.eu%2Fweb%2Fsentinel%2Fmissions%2Fsentinel-2%2Fnews%3Bjsessionid%3D2BC4823847466509FA382BB7ED954C47.jvm1%3Fp_p_id%3D101_INSTANCE_Ac0d%26p_p_lifecycle%3D0%26p_p_state%3Dnormal%26p_p_mode%3Dview%26p_p_col_id%3Dcolumn-1%26p_p_col_count%3D1%26_101_INSTANCE_Ac0d_cur%3D4%26_101_INSTANCE_Ac0d_keywords%3D%26_101_INSTANCE_Ac0d_advancedSearch%3Dfalse%26_101_INSTANCE_Ac0d_delta%3D30%26_101_INSTANCE_Ac0d_andOperator%3Dtrue) allows every spot on Earth to be revisited every five days with the same viewing direction. However, the northern latutides are not imaged during the dark winter months.

Sentinel 2 consists of 2 satellites. First came Sentinel 2A which was launched in 2015. Next came Sentinel 2b in 2017.

Two additional satellites (Sentinel 2C and 2D) were launched in 2024, making a total of four Sentinel-2 satellites.

Sentinel 2 has many bands but the ones we are interested in in this module will be B2, B3, B4 and B8 Blue, green, red and Near Infrared.

| Band | Resolution | Wavelength | Description                        |
|------|------------|------------|------------------------------------|
| B1   | 60 m       | 443 nm     | Ultra Blue (Coastal and Aerosol)   |
| B2   | 10 m       | 490 nm     | Blue                               |
| B3   | 10 m       | 560 nm     | Green                              |
| B4   | 10 m       | 665 nm     | Red                                |
| B5   | 20 m       | 705 nm     | Visible and Near Infrared (VNIR)  |
| B6   | 20 m       | 740 nm     | Visible and Near Infrared (VNIR)  |
| B7   | 20 m       | 783 nm     | Visible and Near Infrared (VNIR)  |
| B8   | 10 m       | 842 nm     | Visible and Near Infrared (VNIR)  |
| B8a  | 20 m       | 865 nm     | Visible and Near Infrared (VNIR)  |
| B9   | 60 m       | 940 nm     | Short Wave Infrared (SWIR)        |
| B10  | 60 m       | 1375 nm    | Short Wave Infrared (SWIR)        |
| B11  | 20 m       | 1610 nm    | Short Wave Infrared (SWIR)        |
| B12  | 20 m       | 2190 nm    | Short Wave Infrared (SWIR)        |


we can also limit the collection in space and time

### Select all images from last summer in Umeå

In [ ]:
# Set the coordinates for umeå as a boundig box.
Umeå = ee.Geometry.Polygon(
    [[[20.0695, 63.8608],
      [20.3986, 63.8608],
      [20.3986, 63.5922],
      [20.0695, 63.5922]]])

# Set the time interval
start_date = '2025-06-01'
end_date = '2025-07-31'

sentinel2 = (ee.ImageCollection('COPERNICUS/S2')
             .filterBounds(Umeå)
             .filterDate(ee.Date(start_date), ee.Date(end_date))
)
sentinel2

Since multiple images will overlap this time and region we need to do a selection on which images we want to display. One way to do it is to select the images with least cloud coverage.

In [ ]:
# Get the least cloudy image
image = sentinel2.sort('CLOUDY_PIXEL_PERCENTAGE').first()

# Get the date of the image
image_date = image.get('system:time_start')
print("Image Date:", ee.Date(image_date).format('YYYY-MM-dd').getInfo())


Let's inspect Umeå during a clear summer day

In [ ]:
# Display the image on the map
Map = geemap.Map()
Map.setCenter(20.2630, 63.8258, 11) # This is the coordinates in long/lat and the zoom level

Map.addLayer(image, {
    'bands': ['B4', 'B3', 'B2'], # red, green, blue
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}, 'RGB')

# Display the map
Map.addLayerControl()
Map

### Task 2
a)
Why do you only see an image on the left side of Umeå? \
b)
Use what you have learned so far and awnser the question. Could you skii in Umeå in December 2023?


# NDVI
NDVI stands for "Normalized Difference Vegetation Index." It is a vegetation index that quantifies the presence and health of vegetation in a given area based on the difference in reflectance between near-infrared and red bands of satellite or aerial imagery. NDVI values typically range -1 to 1, where higher values indicate healthier and more dense vegetation.

 Lets have a closer look at Gotland. First in color. Gotland is to large to fit into one image so we can create a mosaic dataset. Note that this is a speial kind of mosaic where each pixel is selected seperatly from all avalible images within the time window 2025-06-01 to 2025-09-30

In [ ]:
import ee
import geemap

Gotland = ee.Geometry.Polygon([
    [18.743, 57.908],  # NW corner
    [19.726, 57.908],  # NE corner
    [19.726, 57],       # SE corner
    [18.743, 57]        # SW corner
])

START_DATE = '2024-06-01'
END_DATE = '2024-09-30'

# Set cloud filtering parameters
CLOUD_FILTER = 10  # Maximum cloud cover percentage
CLD_PRB_THRESH = 10  # Cloud probability threshold
NIR_DRK_THRESH = 0.15  # Near-infrared dark threshold

sentinel2 = (ee.ImageCollection('COPERNICUS/S2')
             .filterBounds(Gotland)
             .filterDate(ee.Date(START_DATE), ee.Date(END_DATE))
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
             .map(lambda image: image.updateMask(image.select(['B8']).gt(NIR_DRK_THRESH)))
            )

# Create a cloud-free composite
composite = sentinel2.reduce(ee.Reducer.median())

Map = geemap.Map()
Map.addLayer(composite, {
    'bands': ['B4_median', 'B3_median', 'B2_median'], # This selects the median value from all cloud free pixels.
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}, 'RGB Composite')
Map.setCenter(18.4, 57.5, 10)
Map.addLayerControl()
Map


NDVI can be calculated by adding this line of code to the previous code: ndvi = composite.normalizedDifference(['B8_median', 'B4_median'])


In [ ]:
import ee
import geemap

Gotland = ee.Geometry.Polygon([
    [18.743, 57.908],  # NW corner
    [19.726, 57.908],  # NE corner
    [19.726, 57],       # SE corner
    [18.743, 57]        # SW corner
])

START_DATE = '2025-06-01'
END_DATE = '2025-09-30'
CLOUD_FILTER = 10  # Maximum cloud cover percentage
CLD_PRB_THRESH = 10  # Cloud probability threshold
NIR_DRK_THRESH = 0.15  # Near-infrared dark threshold

sentinel2 = (ee.ImageCollection('COPERNICUS/S2')
             .filterBounds(Gotland)
             .filterDate(ee.Date(START_DATE), ee.Date(END_DATE))
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
             .map(lambda image: image.updateMask(image.select(['B8']).gt(NIR_DRK_THRESH)))
            )

composite = sentinel2.reduce(ee.Reducer.median())

# Calculate NDVI
ndvi = composite.normalizedDifference(['B8_median', 'B4_median'])

Map = geemap.Map()
Map.addLayer(ndvi, {
    'palette': ['blue', 'white', 'green'],  # Adjust the palette from RGB to something more fitting NDVI
    'min': -1,
    'max': 1
}, 'NDVI')

Map.setCenter(18.4, 57.5, 15)
Map.addLayerControl()
Map


### Task 3
Calculate NDVI for your home town/village (or some other location you know well) for the summer of 2018.

# Combine satellite images with other data
To analyse the data in satellite imagery we often need to extract it to something we are interested in. It can be field plots, forest stands, wetlands ect. In this case we will use wetlands from Gotland which has been classified based on their "nature value". Start by downloading the wetland polygons as shapefiles from Naturvårdsverket.

In [ ]:
!wget https://geodata.naturvardsverket.se/nedladdning/VMI/ursprunglig_digitalisering/I_Gotland_VMI.zip
!unzip /content/I_Gotland_VMI.zip

Inspect the data in the shapefile

In [ ]:
import geopandas as gpd
gdf = gpd.read_file("/content/I_Gotland_VMI_Ytor.shp")
gdf

This dataset has some issues with naming where letters like å,ö and ä are used. Instead of inspecting all columns and fixing issues we can just select the column we are interested in and drop the rest.

In [ ]:
gdf = gdf[gdf.geom_type == 'Polygon'] # drop all fatures that are not polygons
gdf = gdf[['NVKLASS', 'geometry']].copy() # copy the columns to keep
# 1. Fix invalid polygons
gdf['geometry'] = gdf['geometry'].buffer(0)
gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.0001, preserve_topology=True)

# --- Clean geometries for Earth Engine ---
def clean_geom(g):
    if g is None or g.is_empty:
        return None
    if not g.is_valid:
        g = g.buffer(0)
    if g.geom_type == "Polygon":
        ext = [(float(round(x,6)), float(round(y,6))) for x,y,*_ in g.exterior.coords if abs(x)<=180 and abs(y)<=90]
        return Polygon(ext) if len(ext)>=4 else None
    if g.geom_type == "MultiPolygon":
        polys = [p for p in g.geoms if p.is_valid and not p.is_empty]
        return MultiPolygon(polys) if polys else None
    return None


# Translate the swedish names to english names to avoid symbols like å, ö ,ä.
value_mapping = {'Låga naturvärden':'Low nature values','Vissa naturvärden':'Some nature values', 'Högt naturvärde':'High Nature value', 'Mycket högt naturvärde': 'Very high nature values'}
gdf['NVKLASS'] = gdf['NVKLASS'].replace(value_mapping)
#gdf = gdf.iloc[:100, :]
#save to new shapefile
gdf.to_file("/content/I_Gotland_VMI_Ytor_clean.shp")

Now we can load the cleaned shapefile into geemap by converting it to a format that google earth engine can handle using shp_to_ee. In order to do that you first need to install pycrs

In [ ]:
!pip install pycrs

Lets start by just overlaying the poygons on a map to see if the look ok after our data cleaning

In [ ]:
import geemap
import geopandas as gpd

Map = geemap.Map()
wetlands_shp = "/content/I_Gotland_VMI_Ytor_clean.shp"

# Load and clean
gdf = gpd.read_file(wetlands_shp)
gdf['geometry'] = gdf['geometry'].buffer(0)  # Easy hack to fix broken geometries
gdf = gdf.to_crs('EPSG:4326')  # Convert to WGS84


Map.add_basemap('CartoDB.DarkMatter')
Map.add_gdf(
    gdf,
    layer_name="Wetlands",
    style={
        'color': '#00FFFF',
        'fillColor': '#00FFFF',
        'fillOpacity': 0.3,
        'weight': 2
    }
)

Map.setCenter(18.4, 57.5, 9)
Map.addLayerControl()
Map

# Extract raster values to polygons
Now you have seen how to extract NDVI and how to import polygons to geemap. Lets combine the two and extract NDVI values for these wetlands for further analysis. Start by creating a mosaic of NDVI for Gotland. This may take some time.

In [ ]:
import geopandas as gpd, geemap
from shapely.geometry import Polygon, MultiPolygon

#gdf = gpd.read_file("/content/I_Gotland_VMI_Ytor_clean.shp").to_crs("EPSG:4326")


# # --- Clean geometries for Earth Engine ---
def clean_geom(g):
    if g is None or g.is_empty:
        return None
    if not g.is_valid:
        g = g.buffer(0)
    if g.geom_type == "Polygon":
        ext = [(float(round(x,6)), float(round(y,6))) for x,y,*_ in g.exterior.coords if abs(x)<=180 and abs(y)<=90]
        return Polygon(ext) if len(ext)>=4 else None
    if g.geom_type == "MultiPolygon":
        polys = [p for p in g.geoms if p.is_valid and not p.is_empty]
        return MultiPolygon(polys) if polys else None
    return None

gdf["geometry"] = gdf["geometry"].apply(clean_geom)

# --- Convert to Earth Engine ---
wetlands_ee = geemap.geopandas_to_ee(gdf)

# calculate NDVI
sentinel2 = (ee.ImageCollection('COPERNICUS/S2')
             .filterBounds(Gotland)
             .filterDate(ee.Date(START_DATE), ee.Date(END_DATE))
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
             .map(lambda image: image.updateMask(image.select(['B8']).gt(NIR_DRK_THRESH)))
            )
composite = sentinel2.reduce(ee.Reducer.median())
ndvi = composite.normalizedDifference(['B8_median', 'B4_median'])
ndvi_stats = ndvi.reduceRegions(
    collection=wetlands_ee,
    reducer=ee.Reducer.mean(),
    scale=10
)
features = ndvi_stats.getInfo()['features']
ndvi_gdf = gpd.GeoDataFrame.from_features(features)

We can now properly analyse the data. Here is a quick plot of mean ndvi on wetlands split on nature value.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.violinplot(x='NVKLASS', y='mean', data=ndvi_gdf, palette='Dark2')
plt.title('NDVI from wetlands on Gotland from summer 2023')
plt.xlabel('Nature Class')
plt.ylabel('Mean NDVI')
plt.show()


# Time series
One cool thing with satellites is the temporal aspect which allows us to follow environmental change over time. This requires a bit more work but lets walk through it. Lets limit the analysis to 500 randomly sampled wetlands to save time.

In [ ]:
import geopandas as gpd
gdf = gpd.read_file("/content/I_Gotland_VMI_Ytor_clean.shp")
sampled_geodf = gdf.sample(n=500, random_state=42)
sampled_geodf.to_file("/content/sampled_wetlands.shp")

Next lets start with the first half of the code which is the same as before.

In [ ]:
import ee
import geemap
import geopandas as gpd

# Define Gotland geometry
Gotland = ee.Geometry.Polygon([
  [18.743, 57.908],  # NW corner
  [19.726, 57.908],  # NE corner
  [19.726, 57],      # SE corner
  [18.743, 57]       # SW corner
])

# Set parameters
START_YEAR = 2023
END_YEAR = 2023

# Cloud filtering
CLOUD_FILTER = 10  # Maximum cloud cover percentage
CLD_PRB_THRESH = 10  # Cloud probability threshold
NIR_DRK_THRESH = 0.15  # Near-infrared dark threshold

def filter_monthly_sentinel2(month):
  """Filters Sentinel-2 data for a specific month"""
  start_date = ee.Date(f'{START_YEAR}-{month}-01')
  end_date = start_date.advance(1, 'month')
  return (
      ee.ImageCollection('COPERNICUS/S2')
      .filterBounds(Gotland)
      .filterDate(start_date, end_date)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
      .map(lambda image: image.updateMask(image.select(['QA60']).lt(CLD_PRB_THRESH)))
      .map(lambda image: image.updateMask(image.select(['B8']).gt(NIR_DRK_THRESH)))
  )

# Load wetland shapefile
wetlands_shp = "/content/sampled_wetlands.shp"
gdf = gpd.read_file(wetlands_shp).to_crs("EPSG:4326")
gdf["geometry"] = gdf["geometry"].apply(clean_geom)
wetlands_ee = geemap.geopandas_to_ee(gdf)

Next you need to specify which months that you are interested in and then loop over ndvi mosaics for each month. This step takes a few minutes.

In [ ]:
monthly_ndvi = []
start_month = 1
end_month = 12
month_names = [f"{START_YEAR}-{m:02d}" for m in range(start_month, end_month)]
for month in range(start_month, end_month):
  # Filter Sentinel-2 data for the month
  monthly_images = filter_monthly_sentinel2(month)

  # Calculate monthly NDVI composite
  monthly_composite = monthly_images.reduce(ee.Reducer.median())
  monthly_ndvi.append(monthly_composite.normalizedDifference(['B8_median', 'B4_median']))

# Extract NDVI statistics per wetland for each month
ndvi_stats_list = []
for i, ndvi_image in enumerate(monthly_ndvi):
  ndvi_stats = ndvi_image.reduceRegions(
      collection=wetlands_ee,
      reducer=ee.Reducer.mean(),  # You can change the reducer as needed
      scale=10
  )
  ndvi_stats_list.append(ndvi_stats.getInfo()['features'])

# Combine monthly NDVI stats into a single GeoDataFrame
all_features = []
for month_data in ndvi_stats_list:
  all_features.extend(month_data)

ndvi_gdf = gpd.GeoDataFrame.from_features(all_features)



# Add a column for month
ndvi_gdf['month'] = None  # Initialize the month column

for i, row in ndvi_gdf.iterrows():
  # Use modulo (%) to handle cases with more months than data points
  month_index = i % len(month_names)
  ndvi_gdf.at[i, 'month'] = month_names[month_index]

ndvi_gdf

Now we have a dataframe to analyse further. Time series of NDVI values can be used as input for machine learning models to extract spatiotemporal patterns from the data.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.boxplot(x="month", y="mean", hue='NVKLASS', data=ndvi_gdf, palette='Dark2')
plt.title('NDVI in wetlands with very high nature value')
plt.xlabel('Month')
plt.ylabel('Mean NDVI')
plt.show()

Since this is temporal data we can reuse some code from assignment three to create animated plots. Start by installing chart studio.

In [ ]:
!pip install chart_studio

Notice how the shapes changes over time and pay attention to the behaviour of Wetlands with low nature values compared to the others.

In [ ]:
import plotly.express as px

df = px.data.gapminder()

fig = px.violin(ndvi_gdf, x="NVKLASS", y="mean", color="NVKLASS",
  animation_frame="month", animation_group="NVKLASS", range_y=[-0.2,1],template="plotly_dark")


fig.show()

### Task 4
Select another part of Sweden and see if the wetlands behave differently from wetlands on Gotland based on NDVI values.

you will find other parts of Sweden here: https://geodata.naturvardsverket.se/nedladdning/VMI/ursprunglig_digitalisering/

If the datasize is to big you can randomly sample fewer wetlands. Remember that we used about 500 wetlands so far. Qgis is a good companion to notebooks. It's always a good habbit to inspect the data you are working with.

# Change detecton

In [ ]:
import ee
import geemap


# (Gotland)
Gotland = ee.Geometry.Polygon([
  [18.743, 57.908],
  [19.726, 57.908],
  [19.726, 57.0],
  [18.743, 57.0]
])

START_1 = '2020-06-01'
END_1   = '2020-08-31'
START_2 = '2024-06-01'
END_2   = '2024-08-31'
CLOUD_FILTER = 20

def get_s2_composite(start, end):
    """Return median Sentinel-2 composite with NDVI band."""
    collection = (
        ee.ImageCollection('COPERNICUS/S2')
        .filterBounds(Gotland)
        .filterDate(start, end)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
    )
    composite = collection.median()
    ndvi = composite.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi

# Compute NDVI
ndvi_2020 = get_s2_composite(START_1, END_1)
ndvi_2024 = get_s2_composite(START_2, END_2)
ndvi_change = ndvi_2024.subtract(ndvi_2020).rename('NDVI_change')

change_vis = {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'blue']} #Blue → vegetation increase (higher NDVI in 2024). Red → vegetation loss (lower NDVI in 2024)

Map = geemap.Map(center=[57.4, 18.9], zoom=8)
Map.addLayer(ndvi_change, change_vis, 'NDVI Change (2024-2020)')
Map.add_colorbar(change_vis['palette'], vmin=-0.5, vmax=0.5, label='NDVI Change')
Map


# Watch the world burn
### Inspect the area before fires

In [ ]:
import ee
import geemap


# --- Region of Interest: Ourense, Galicia (Spain wildfires 2025) ---
roi = ee.Geometry.Polygon([
  [-7.8, 42.6],
  [-7.0, 42.6],
  [-7.0, 42.2],
  [-7.8, 42.2]
])

# --- Pre-fire time period ---
pre_start = '2025-06-01'
pre_end   = '2025-06-30'
CLOUD_FILTER = 20

# --- Get Sentinel-2 composite and compute NBR ---
def get_nbr_composite(start, end):
    s2 = (ee.ImageCollection('COPERNICUS/S2')
          .filterBounds(roi)
          .filterDate(start, end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
          .median())
    nbr = s2.normalizedDifference(['B8', 'B12']).rename('NBR')
    return nbr

nbr_pre = get_nbr_composite(pre_start, pre_end)

# --- Visualization parameters ---
nbr_vis = {'min': 0, 'max': 1, 'palette': ['brown', 'yellow', 'green']}

# --- Display ---
Map = geemap.Map(center=[42.4, -7.4], zoom=9)
Map.addLayer(nbr_pre, nbr_vis, 'NBR Pre-fire (June 2025)')
Map


### After fires

In [ ]:
# --- Post-fire period ---
post_start = '2025-08-15'
post_end   = '2025-09-15'

nbr_post = get_nbr_composite(post_start, post_end)

# --- Compute delta NBR (burn severity) ---
dnbr = nbr_pre.subtract(nbr_post).rename('dNBR')

# --- Visualization ---
nbr_vis_post = {'min': 0, 'max': 1, 'palette': ['brown', 'yellow', 'green']}
dnbr_vis     = {'min': 0, 'max': 1, 'palette': ['white', 'yellow', 'orange', 'red', 'black']}

Map = geemap.Map(center=[42.4, -7.4], zoom=9)
Map.addLayer(nbr_post, nbr_vis_post, 'NBR Post-fire (Aug–Sep 2025)')
#Map.addLayer(dnbr, dnbr_vis, 'dNBR Burn Severity')
Map.add_colorbar(dnbr_vis['palette'], vmin=0, vmax=1, label='Burn Severity (dNBR)')
Map


## Task 5 Monitoring La Palma Volcanic Eruption
There was an eruption at La Palma 2021-09-19. Your task is to Use sentinel 2 imagery to detect vegetation loss and burned/lava-affected areas using NDVI and NBR from Sentinel-2 imagery. The eruption lasted for two months. \
a) Inspect NDVI change: Identify areas with strong vegetation loss.\
b) Inspect NBR change: Identify lava and burned areas.

# Spy on the Russian navy or watch the snow melt
This segment is only for inspiration and there are no more tasks to complete in this module.



## Ship detection
Russian warships from the Soviet era have red decks which makes them easy to pick out using NDVI, even from low resolution sentinel2 imagery.

In [ ]:
import ee
import geemap

# Severomorsk
roi = ee.Geometry.Polygon([
    [33.39, 69.01],
    [33.47, 69.01],
    [33.47, 69.09],
    [33.39, 69.09]
])
ndwi = image.normalizedDifference(['B3','B8']).rename('NDWI')

water_mask = ndwi.gt(-0.2)
# Get a single Sentinel-2 image
image = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterBounds(roi)
          .filterDate('2024-06-01', '2024-07-31')
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
          .sort('CLOUDY_PIXEL_PERCENTAGE')
          .first())

# Detect red pixels: B4 (red) > 0.1
red_index = image.normalizedDifference(['B4','B3']).rename('Red_Index')
bright_red_mask = image.select('B4').divide(10000).gt(0.1)
ships_candidates = bright_red_mask.And(red_index.gt(0.1)).updateMask(water_mask)

# Visualization
rgb = image.select(['B4','B3','B2']).divide(10000)
Map = geemap.Map(center=[69.085, 33.43], zoom=15)
Map.addLayer(rgb, {'min':0, 'max':0.3}, 'True color')
Map.addLayer(ships_candidates.updateMask(ships_candidates), {'palette':['red']}, 'Red Candidate Pixels')
Map


### Snow monitor
While developing this module I also built a side project for my kids where they can follow the spring from space using satellite imagery. It's a small script that runs on a Rasberry pi and simply pulls the latest satellite imagery of our village every day and displays it on a old monitor that I set up in our kitchen. You can use this for anything from onitoring ships in an haurbor or active vulcanos. This is what you need:





*   Some easily impressed kids.
*   An old monitor.
*   An old computer or something equalent to Rasberry pi.
*   An idea of what to spy on.

Modify the code bellow so the coordiantes covers the area you want to spy on.
Copy the code and save it to a script that you call something like "spy.py" Run the scipt. The while loop will look for the latest satellite imagery once every day and then display that image on the monitor.

In [ ]:
import ee
import geemap
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time

# Authenticate and initialize Google Earth Engine API
ee.Authenticate()

# Change 'my-project' to your actual project
ee.Initialize(project='ee-williamlidberg')

# Set the location (bounding box)
roi = ee.Geometry.Rectangle([19.75, 63.9, 20, 64.03])

# Define the number of days to look back
num_days_to_look_back = 5
plt.figure(figsize=(20, 20))

while True:  # Run indefinitely
    latest_image = None
    latest_acquisition_time = None
    for i in range(num_days_to_look_back):
        start_date = datetime.today() - timedelta(days=i + 1)
        end_date = datetime.today() - timedelta(days=i)
        collection = (ee.ImageCollection('COPERNICUS/S2')
                      .filterBounds(roi)
                      .filterDate(start_date, end_date)
                      .sort('system:time_start', False)  # Sort in descending order to get the latest image first
                      )
        # Get the latest image for the current day
        image = collection.first()
        if image.getInfo():
            image_array = geemap.ee_to_numpy(image.select(['B4', 'B3', 'B2']).divide(4000), region=roi)
            if image_array.size > 0:
                latest_image = image
                latest_acquisition_time = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss').getInfo()
    # Check if the latest image and acquisition time are available
    if latest_image is not None and latest_acquisition_time is not None:
        center = roi.centroid().coordinates().reverse().getInfo()
        zoom = 12  # Adjust the zoom level as needed

        plt.clf()
        image_array = geemap.ee_to_numpy(latest_image.select(['B4', 'B3', 'B2']).divide(4000), region=roi)
        if image_array.size > 0:
            plt.imshow(image_array, vmin=0, vmax=0.4, aspect='equal')
            plt.title(f'True Color Image - {latest_acquisition_time}')
            plt.axis('off')

        print(f'Acquisition Time of the Latest Image: {latest_acquisition_time}')
        plt.pause(1)
        print('Updated image')

    time.sleep(86400) # number of seconds in a day


## References
  * Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
  * Wu, Q., Lane, C. R., Li, X., Zhao, K., Zhou, Y., Clinton, N., DeVries, B., Golden, H. E., & Lang, M. W. (2019). Integrating LiDAR data and multi-temporal aerial imagery to map wetland inundation dynamics using Google Earth Engine. Remote Sensing of Environment, 228, 1-13. https://doi.org/10.1016/j.rse.2019.04.015 (pdf | source code)